In [14]:
import numpy as np
from itertools import combinations, chain
from scipy.spatial.distance import cdist
from scipy.io import loadmat

In [15]:
def reduction(homology_basis, w, wlowv, v, support_v):
    if not w:#if col_wo is empty after removing lowv then multiply -(value of lowv) to all other simplices in v and return
        return {tau: (-wlowv * val) % homology_basis['coeff_field'] for tau, val in v.items()} 
        #return dict((tau, (- wlowv * v[tau]) 
        #                  % homology_basis['coeff_field']) for tau in support_v)
    v_multiplied = {tau: -wlowv * val for tau, val in v.items()}
    #new_v = {tau: val for tau, val in v_multiplied.items() if tau in w.keys()}
    red = w.copy()
    red.update(v_multiplied)
    for tau in support_v.intersection(w.keys()):
        redtau = (w.get(tau, 0) + v_multiplied.get(tau,0)) % homology_basis['coeff_field'] #column operations for reduction
        if redtau == 0:
            red.pop(tau)
        else:
            red[tau] = redtau
    return red #dict((tau, val) for tau, val in red.items() if val > 0) 

In [16]:
def kill_persistence(homology_basis, v, simplex):
    homology_basis['v'].append(v)
    support_v = set(v) 
    lowv = homology_basis['simplices'][max(homology_basis['index_table'][face] for face in support_v)]
    support_v.remove(lowv) #why do we remove lowv from support
    inversevlowv = homology_basis['inversion_function'](v[lowv])
    v.pop(lowv)
    v = {tau: (val * inversevlowv) % #####################################tau are faces of simplex and they are paired                                                              
         homology_basis['coeff_field'] for tau, val in v.items()}         #with their value in the old homology basis
    homology_basis['persistence'][lowv] = simplex

    
    #print('transpose', homology_basis['transpose'])
    #print('basis', homology_basis['basis'])
    for column, val in homology_basis['transpose'].pop(lowv):
        column_without_lowv = dict(homology_basis['basis'][column])
        print(lowv, column_without_lowv)
        column_without_lowv.pop(lowv)
        reduced_column = reduction(
            homology_basis, #homology basis
            column_without_lowv, #removal of lowv simplex
            val, 
            v, 
            support_v)
#        , 
#            inversevlowv)
        homology_basis['basis'][column] = frozenset(reduced_column.items())
        for row in support_v.intersection(column_without_lowv):
            homology_basis['transpose'][row].remove((column, column_without_lowv[row]))
        
        if not homology_basis['basis'][column]:
            homology_basis['basis'].pop(column)
            homology_basis['keys'].remove(column)
            #homology_basis['row_support'].pop(column, None)
            homology_basis['index_table'].pop(column)
        else:
            for row in support_v.intersection(reduced_column):
                homology_basis['transpose'][row].add((column, reduced_column[row]))

        
def contract_homology_basis(
    homology_basis,
    simplex):
    
    boundary = tuple(combinations(simplex, len(simplex) - 1))
    v = dict()
    for face in set.intersection(set(boundary), homology_basis['keys']):
        if not homology_basis['basis'].get(face): #Apply when face is not in 'basis'
            homology_basis['basis'][face] = frozenset(((face, 1),)) #Adds face to 'basis'
            homology_basis['transpose'][face] = set(((face, 1),)) #Adds face to 'transpose'
        for h_face, h_value in homology_basis['basis'][face]: ##############What exactly is h_val in persistence homology?
            
            #v not equal to zero (death) check from theorem 1.9
            v_value = (v.pop(h_face, 0) + ##################################################what does v.pop(h_face, 0) do?
                       (-1)**boundary.index(face) * h_value) % homology_basis['coeff_field']
            if v_value > 0:#####################################################This checks if the value in the old 
                v[h_face] = v_value                                             #homology basis non zero.
        
    if v:
        kill_persistence(homology_basis, v, simplex)
        return True
    else:
        return False


def extend_homology_basis(
    homology_basis,
    simplex):
        
        
    if not contract_homology_basis(homology_basis, simplex):
        
        homology_basis['length'] += 1
        homology_basis['keys'].add(simplex)
        homology_basis['index_table'][simplex] = homology_basis['length']
        homology_basis['simplices'][homology_basis['length']] = simplex
        


In [17]:
def field_inversion(coeff_field):
    def inverse(a, n):
        for x in range(n):
            if (a * x) % n == 1:
                return x
    inversion_table = [
        inverse(a + 1, coeff_field) for a in range(coeff_field - 1)]
    def inversion_fct(a):
        return inversion_table[a-1]
    return inversion_fct

In [18]:
def filtered_homology_basis(ordered_complex, max_face_card, coeff_field=11):
    homology_basis = dict((
        ('v', list()),
        ('basis', dict()),
        ('transpose', dict()),
        ('length', 0), 
        ('index_table', dict()),
        ('persistence', dict()), 
        ('keys', set()), 
        #('row_support', dict()),
        ('coeff_field', coeff_field),
        ('inversion_function', field_inversion(coeff_field)),
        ('simplices', dict())
    ))
    
    for face in ordered_complex:
        if len(face) < max_face_card:
            extend_homology_basis(homology_basis, face)
        else:
            contract_homology_basis(homology_basis, face)
    return homology_basis

In [19]:
def persistence_pairs(
    homology_basis,
    filtration_function,
    max_dimension):
    #index_table = {face: index for index, face in enumerate(simplicial_complex)}
    persistent_homology_pairs = dict()
    persistent_homology_pairs[0] = [(0, np.inf)]
    for birth, death in homology_basis['persistence'].items():
        pairs = persistent_homology_pairs.get(
            len(birth) - 1, list())
        pairs.append((
            filtration_function(birth), filtration_function(death)))
        persistent_homology_pairs[len(birth) - 1] = pairs
    persistent_homology_pairs = {dim: np.array(pairs) for dim, pairs in persistent_homology_pairs.items()}
    persistent_homology_pairs = {dim: pairs[pairs[:,0] + 5*np.finfo(float).eps < pairs[:,1]]
                                 for dim, pairs in persistent_homology_pairs.items()}
    persistent_homology_pairs = {dim: pairs[np.argsort(
        pairs[:,0] - pairs[:,1])] for dim, pairs in persistent_homology_pairs.items()}#####################################
    persistent_homology_pairs = {dim: pairs for dim, pairs in persistent_homology_pairs.items() if len(pairs) > 0}#########
    for dim in range(max_dimension):
        persistent_homology_pairs.setdefault(dim, np.empty((0,2))) 
    return persistent_homology_pairs